In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

path = "drive/MyDrive/Capstone/"
crop_1 = "Barley"
crop_2 = "Maize"
crop_6 = "Sorghum"
crop_7 = "Teff"
crop_8 = "Wheat"

year = "2011"

df = pd.read_excel(f"{path}Meher_{year}.xlsx")

In [6]:
df.columns


Index(['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area',
       'Household Id', 'Head sex', 'Holder id', 'Parcel', 'Field',
       'Sampling Weight', 'Field Type', 'CROP', 'Ownership',
       'Is field under Extension Program?', 'Is Field Irrigated?',
       'If Field Irrigated source of  water',
       'Is Field Prevented form Erosion', 'Common way of prevention',
       'Number of Fruit Trees', 'Number of Fruit Bearing Trees',
       'Seed / Seedling Type', 'Quantity of improved seeds used',
       'Price of improved seeds used', 'Quantity of indigenous seeds used',
       'Was crop damaged?', 'If yes, cause of damage',
       'Percent of damaged crop', 'Prevension/precaution measure taken?',
       'Type of measure', 'Chemical type used if any', 'Is Fertilizer Used?',
       'Type of fertilizer used if any?', 'If chemical fertilizer used',
       'Quantity of chemical fertilizer used',
       'If natural fertilizer used, type',
       'How often is temporary c

In [7]:
previous_feature = 'What was the previous state of the field ?'

if year == "2010":
  previous_feature = "What was the field used for?"

if year == "2013":
  previous_feature = "D26"

if year in ("2011", "2012"):
  previous_feature = "What was the previous state of the field?"

df['What was the previous state of the field?'] = df[previous_feature]
previous_feature = 'What was the previous state of the field?'

In [8]:
damage_feature = 'Percent of damaged crop'

if year == "2010":
  damage_feature = 'Damage in Percent' 

if year == "2013":
  damage_feature = 'DPERCENT'

df['Damage in Percent'] = df[damage_feature].fillna(0)
damage_feature = 'Damage in Percent'

In [9]:
df[damage_feature].describe()

count    144590.000000
mean         16.898153
std          33.296643
min           0.000000
25%           0.000000
50%           0.000000
75%          30.000000
max         999.000000
Name: Damage in Percent, dtype: float64

In [10]:
df[df[damage_feature] > 100] = 100

In [11]:
df[damage_feature].describe()

count    144590.000000
mean         16.450488
std          25.131801
min           0.000000
25%           0.000000
50%           0.000000
75%          30.000000
max         100.000000
Name: Damage in Percent, dtype: float64

In [12]:
fertilizer_qty_features = []
       
if year == "2010":
  fertilizer_qty_features =['If Chemical Fertilizer,Quantity in KG']

if year == "2011":
  fertilizer_qty_features = ['Quantity of chemical fertilizer used']

if year == "2012":  
  fertilizer_qty_features = ['Quantity of UREA chemical fertilizer used',
       'Quantity of DAP chemical fertilizer used',
       'Quantity of Both UREA and DAP chemical fertilizer used']

if year == "2013":
  fertilizer_qty_features = ["D22B", "D22C", "D22D", "D22E"]

if year in ("2014", "2015"):
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',]

if year == "2016":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',
       'Quantity of OTHER fertilizer used',]

if year == "2017":
  fertilizer_qty_features = [ 'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',
       'Quantity of OTHER fertilizer used',
       'Quantity of UREA IF both UREA and NPS are used',
       'Quantity of NPS IF both UREA and NPS are used',]
       
if year == "2018":
  fertilizer_qty_features = [  'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of UREA fertilizer used, if both DAP and UREA are used',
       'Quantity of DAP fertilizer used, if both DAP and UREA are used',
       'Quantity of NPS fertilizer used',
       'Quantity of UREA IF both UREA and NPS are used',
       'Quantity of NPS IF both UREA and NPS are used',
       'Quantity of OTHER fertilizer used', ]
              
if year == "2019":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of NPS fertilizer used,', 'Quantity of MIXED fertilizer used',
       'If natural fertilizer used, type', ]

if year == "2020":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
       'Quantity of NPS fertilizer used,',
       'Quantity of BLENDED fertilizer used',]

if year == "2021":
  fertilizer_qty_features = ['Quantity of UREA fertilizer used',
       'Quantity of DAP fertilizer used', 'Quantity of NPS fertilizer used,',
       'Quantity of BLENDED fertilizer used',]
       
fertilizer_qty_features

['Quantity of chemical fertilizer used']

In [13]:
df[fertilizer_qty_features] = df[fertilizer_qty_features].fillna(0)

In [14]:
df["Fertilizer Quantity"] = df[fertilizer_qty_features].sum(axis=1)

In [15]:
df["Fertilizer Quantity"].describe()

count    144590.000000
mean          8.168631
std          26.125552
min           0.000000
25%           0.000000
50%           0.000000
75%           6.600000
max        3882.260000
Name: Fertilizer Quantity, dtype: float64

In [16]:
fert_threshold = df["Fertilizer Quantity"].quantile(.995)
fert_threshold

116.468

In [17]:
df = df[df["Fertilizer Quantity"] < fert_threshold]

In [18]:
df.shape

(143539, 44)

In [20]:
df_prev = df[previous_feature]
df = df.dropna(axis=1, thresh=df.shape[0]/2)
df[previous_feature] = df_prev
df.shape

(143539, 33)

In [189]:
df.columns

Index(['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area',
       'Household Id', 'Head sex', 'Holder id', 'Parcel', 'Field',
       'Sampling Weight', 'Field Type', 'CROP', 'Ownership',
       'Is field under Extension Program?', 'Is Field Irrigated?',
       'Is Field Prevented form Erosion', 'Common way of prevention',
       'Seed / Seedling Type', 'Quantity of indigenous seeds used',
       'Was crop damaged?', 'Prevension/precaution measure taken?',
       'Type of measure', 'Is Fertilizer Used?',
       'Type of fertilizer used if any?',
       'Quantity of chemical fertilizer used',
       'How often is temporary crop field used  in  Meher (main) season?',
       'What was the previous state of the field?', 'Area in Hectare',
       'LANDUSE', 'PRODUCTION IN QUINTALS', 'Damage in Percent',
       'Fertilizer Quantity'],
      dtype='object')

In [190]:
area = "Area in Square meter"
production = "Production Of Condition in Kg"
crop = "CROP"
region = "Region"
district = "District"

if year == "2013":
  area = "AREA"

if year == "2012":
  area = "Computed Area"
  production = "PRODUCTION"
  
if year == "2011":
  area = "Area in Hectare"
  production = "PRODUCTION IN QUINTALS"
  
if year == "2010":
  area = "AREAH"
  production = "PRODUCTION IN QUINTALS"
  
if year == "2009":
  area = "Area in hectare"
  production = "Production in Quintal"
  crop = "Crop or Landuse"
  
if year == "2008":
  area = "Area (sq. m.)"
  production = "Production in Quintal"
  crop = "Crop or Land Use"
  
if year == "2007":
  area = "Area in Hectare"
  production = "Production in Quintal"
  
if year == "2006":
  region = "Killil"
  district = "Wereda"
  area = "Area in hectar"
  production = "Production in Quintal"
    
if year == "2005":
  region = "Killil"
  district = "Wereda"
  area = "Area (sq. m.)"
  production = "Production in Quintal"
  crop = "Crop or Land Use"
    
if year == "2004":
  district = "District(Wereda)"
  area = "Area (sq. m.)"
  production = "Dry Weight Production (kg.)"
  crop = "Crop or Land Use"
    


In [191]:
df[[area]].describe()
df[[production]].describe()

,PRODUCTION IN QUINTALS
count,143487.000000
mean,3.347435
std,5.789759
min,0.000000
25%,0.544900
50%,1.650400
75%,4.029400
max,323.838600


In [192]:
min_threshold, max_threshold = df[[area]].quantile([0.045, 0.955]).values[:, 0]
min_threshold, max_threshold

(0.002780639999999999, 0.6280749199999996)

In [193]:
df = df[(df[area] > min_threshold) & (df[area] < max_threshold)]

In [194]:
df.shape

(130583, 33)

In [195]:
df_valid = df.dropna(subset=[region, "Zone", district, crop, area, production])
df_valid.shape

(130571, 33)

In [196]:
df.nunique()

Region                                                                  10
Zone                                                                    21
District                                                                24
Farmers Association                                                     71
Enumeration Area                                                        15
Household Id                                                           410
Head sex                                                                 2
Holder id                                                                9
Parcel                                                                  59
Field                                                                   52
Sampling Weight                                                       1380
Field Type                                                               2
CROP                                                                     5
Ownership                

In [197]:
categorical_unique_value_limit = 100

df_zero = df_valid.fillna(0)
categorical_cols = []
numerical_cols = []

for col in df_zero.columns:
  if np.array_equal(df_zero[col], df_zero[col].astype("int")) and df[col].nunique() < categorical_unique_value_limit:
    categorical_cols.append(col)

    
for col in df_zero.columns:
  if not ( np.array_equal(df_zero[col], df_zero[col].astype("int")) and df[col].nunique() < categorical_unique_value_limit):
    numerical_cols.append(col)


print("Categorical Values (Nominal)")
print(categorical_cols, end="\n\n")

print("Categorical Values (Nominal)")
print(numerical_cols)


Categorical Values (Nominal)
['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area', 'Head sex', 'Holder id', 'Parcel', 'Field', 'Field Type', 'CROP', 'Ownership', 'Is field under Extension Program?', 'Is Field Irrigated?', 'Is Field Prevented form Erosion', 'Common way of prevention', 'Seed / Seedling Type', 'Was crop damaged?', 'Prevension/precaution measure taken?', 'Type of measure', 'Is Fertilizer Used?', 'Type of fertilizer used if any?', 'How often is temporary crop field used  in  Meher (main) season?', 'What was the previous state of the field?', 'LANDUSE', 'Damage in Percent']

Categorical Values (Nominal)
['Household Id', 'Sampling Weight', 'Quantity of indigenous seeds used', 'Quantity of chemical fertilizer used', 'Area in Hectare', 'PRODUCTION IN QUINTALS', 'Fertilizer Quantity']


In [198]:
df_valid.isna().sum()

Region                                                                  0
Zone                                                                    0
District                                                                0
Farmers Association                                                     0
Enumeration Area                                                        0
Household Id                                                            0
Head sex                                                                0
Holder id                                                               0
Parcel                                                                  0
Field                                                                   0
Sampling Weight                                                         0
Field Type                                                              0
CROP                                                                    0
Ownership                             

In [199]:
from sklearn.impute import SimpleImputer

imputer = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                    ("pandarizer",FunctionTransformer(lambda x: pd.DataFrame(x, columns = categorical_cols)))])
df_categorical = imputer.fit_transform(df_valid[categorical_cols])

imputer = Pipeline([("imputer", SimpleImputer(strategy="mean")),
                    ("pandarizer",FunctionTransformer(lambda x: pd.DataFrame(x, columns = numerical_cols)))])
df_numerical = imputer.fit_transform(df_valid[numerical_cols])

df_clean = df_categorical.join(df_numerical)

df_clean.isna().sum()

Region                                                              0
Zone                                                                0
District                                                            0
Farmers Association                                                 0
Enumeration Area                                                    0
Head sex                                                            0
Holder id                                                           0
Parcel                                                              0
Field                                                               0
Field Type                                                          0
CROP                                                                0
Ownership                                                           0
Is field under Extension Program?                                   0
Is Field Irrigated?                                                 0
Is Field Prevented f

In [200]:
if year == "2013":
  df_clean.rename(columns = {'AREA':'Area in Square meter'}, inplace = True)
  df_clean

if year == "2012":
  df_clean.rename(columns = {'Computed Area':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'PRODUCTION':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] / 1000)
  df_clean

if year == "2011":
  df_clean.rename(columns = {'Area in Hectare':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'PRODUCTION IN QUINTALS':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2010":
  df_clean.rename(columns = {'AREAH':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'PRODUCTION IN QUINTALS':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2009":
  df_clean.rename(columns = {'Crop or Landuse':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area in hectare':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2008":
  df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean

if year == "2007":
  df_clean.rename(columns = {'Area in Hectare':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2006":
  df_clean.rename(columns = {'Killil':'Region'}, inplace = True)
  df_clean.rename(columns = {'Wereda':'District'}, inplace = True)
  df_clean.rename(columns = {'Area in hectar':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
  df_clean

if year == "2005":
  df_clean.rename(columns = {'Killil':'Region'}, inplace = True)
  df_clean.rename(columns = {'Wereda':'District'}, inplace = True)
  df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
  df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
  df_clean

if year == "2004":
  df_clean.rename(columns = {'District(Wereda)':'District'}, inplace = True)
  df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
  df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
  df_clean.rename(columns = {'Dry Weight Production (kg.)':'Production Of Condition in Kg'}, inplace = True)
  df_clean


In [201]:
df_1 = df_clean[df_clean["CROP"] == 1]
df_2 = df_clean[df_clean["CROP"] == 2]
df_6 = df_clean[df_clean["CROP"] == 6]
df_7 = df_clean[df_clean["CROP"] == 7]
df_8 = df_clean[df_clean["CROP"] == 8]

print(crop_1, df_1.shape)
print(crop_2, df_2.shape)
print(crop_6, df_6.shape)
print(crop_7, df_7.shape)
print(crop_8, df_8.shape)
print("\nTotal:", df_1.shape[0] + df_2.shape[0] + df_6.shape[0] + df_7.shape[0] + df_8.shape[0])

Barley (17982, 33)
Maize (37689, 33)
Sorghum (27414, 33)
Teff (28395, 33)
Wheat (19091, 33)

Total: 130571


In [202]:
df_1.to_csv(f"{path}Clean_Meher_{year}_{crop_1}.csv")
df_2.to_csv(f"{path}Clean_Meher_{year}_{crop_2}.csv")
df_6.to_csv(f"{path}Clean_Meher_{year}_{crop_6}.csv")
df_7.to_csv(f"{path}Clean_Meher_{year}_{crop_7}.csv")
df_8.to_csv(f"{path}Clean_Meher_{year}_{crop_8}.csv")

In [203]:
print(year)
print(2*f"{80*'* '}\n")

2011
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 



In [204]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

path = "drive/MyDrive/Capstone/"
crop_1 = "Barley"
crop_2 = "Maize"
crop_6 = "Sorghum"
crop_7 = "Teff"
crop_8 = "Wheat"

for year_int in range (2010, 2022):

  year = str(year_int)

  df = pd.read_excel(f"{path}Meher_{year}.xlsx")
  df.columns

  damage_feature = 'Percent of damaged crop'

  if year == "2010":
    damage_feature = 'Damage in Percent' 

  if year == "2013":
    damage_feature = 'DPERCENT'

  df['Damage in Percent'] = df[damage_feature].fillna(0)
  damage_feature = 'Damage in Percent'
  df[damage_feature].describe()
  df[df[damage_feature] > 100] = 100
  df[damage_feature].describe()
  fertilizer_qty_features = []
        
  if year == "2010":
    fertilizer_qty_features =['If Chemical Fertilizer,Quantity in KG']

  if year == "2011":
    fertilizer_qty_features = ['Quantity of chemical fertilizer used']

  if year == "2012":  
    fertilizer_qty_features = ['Quantity of UREA chemical fertilizer used',
        'Quantity of DAP chemical fertilizer used',
        'Quantity of Both UREA and DAP chemical fertilizer used']

  if year == "2013":
    fertilizer_qty_features = ["D22B", "D22C", "D22D", "D22E"]

  if year in ("2014", "2015"):
    fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
        'Quantity of UREA fertilizer used, if both DAP and UREA are used',
        'Quantity of DAP fertilizer used, if both DAP and UREA are used',]

  if year == "2016":
    fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
        'Quantity of UREA fertilizer used, if both DAP and UREA are used',
        'Quantity of DAP fertilizer used, if both DAP and UREA are used',
        'Quantity of OTHER fertilizer used',]

  if year == "2017":
    fertilizer_qty_features = [ 'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
        'Quantity of UREA fertilizer used, if both DAP and UREA are used',
        'Quantity of DAP fertilizer used, if both DAP and UREA are used',
        'Quantity of OTHER fertilizer used',
        'Quantity of UREA IF both UREA and NPS are used',
        'Quantity of NPS IF both UREA and NPS are used',]
        
  if year == "2018":
    fertilizer_qty_features = [  'Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
        'Quantity of UREA fertilizer used, if both DAP and UREA are used',
        'Quantity of DAP fertilizer used, if both DAP and UREA are used',
        'Quantity of NPS fertilizer used',
        'Quantity of UREA IF both UREA and NPS are used',
        'Quantity of NPS IF both UREA and NPS are used',
        'Quantity of OTHER fertilizer used', ]
                
  if year == "2019":
    fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
        'Quantity of NPS fertilizer used,', 'Quantity of MIXED fertilizer used',
        'If natural fertilizer used, type', ]

  if year == "2020":
    fertilizer_qty_features = ['Quantity of UREA fertilizer used', 'Quantity of DAP fertilizer used',
        'Quantity of NPS fertilizer used,',
        'Quantity of BLENDED fertilizer used',]

  if year == "2021":
    fertilizer_qty_features = ['Quantity of UREA fertilizer used',
        'Quantity of DAP fertilizer used', 'Quantity of NPS fertilizer used,',
        'Quantity of BLENDED fertilizer used',]
        
  fertilizer_qty_features
  df[fertilizer_qty_features] = df[fertilizer_qty_features].fillna(0)
  df["Fertilizer Quantity"] = df[fertilizer_qty_features].sum(axis=1)
  df["Fertilizer Quantity"].describe()
  fert_threshold = df["Fertilizer Quantity"].quantile(.995)
  fert_threshold
  df = df[df["Fertilizer Quantity"] < fert_threshold]
  df.shape
  df = df.dropna(axis=1, thresh=df.shape[0]/2)
  df.shape
  df.columns
  area = "Area in Square meter"
  production = "Production Of Condition in Kg"
  crop = "CROP"
  region = "Region"
  district = "District"

  if year == "2013":
    area = "AREA"

  if year == "2012":
    area = "Computed Area"
    production = "PRODUCTION"
    
  if year == "2011":
    area = "Area in Hectare"
    production = "PRODUCTION IN QUINTALS"
    
  if year == "2010":
    area = "AREAH"
    production = "PRODUCTION IN QUINTALS"
    
  if year == "2009":
    area = "Area in hectare"
    production = "Production in Quintal"
    crop = "Crop or Landuse"
    
  if year == "2008":
    area = "Area (sq. m.)"
    production = "Production in Quintal"
    crop = "Crop or Land Use"
    
  if year == "2007":
    area = "Area in Hectare"
    production = "Production in Quintal"
    
  if year == "2006":
    region = "Killil"
    district = "Wereda"
    area = "Area in hectar"
    production = "Production in Quintal"
      
  if year == "2005":
    region = "Killil"
    district = "Wereda"
    area = "Area (sq. m.)"
    production = "Production in Quintal"
    crop = "Crop or Land Use"
      
  if year == "2004":
    district = "District(Wereda)"
    area = "Area (sq. m.)"
    production = "Dry Weight Production (kg.)"
    crop = "Crop or Land Use"
      

  df[[area]].describe()
  df[[production]].describe()
  min_threshold, max_threshold = df[[area]].quantile([0.045, 0.955]).values[:, 0]
  min_threshold, max_threshold
  df = df[(df[area] > min_threshold) & (df[area] < max_threshold)]
  df.shape
  df_valid = df.dropna(subset=[region, "Zone", district, crop, area, production])
  df_valid.shape
  df.nunique()
  categorical_unique_value_limit = 100

  df_zero = df_valid.fillna(0)
  categorical_cols = []
  numerical_cols = []

  for col in df_zero.columns:
    if np.array_equal(df_zero[col], df_zero[col].astype("int")) and df[col].nunique() < categorical_unique_value_limit:
      categorical_cols.append(col)

      
  for col in df_zero.columns:
    if not ( np.array_equal(df_zero[col], df_zero[col].astype("int")) and df[col].nunique() < categorical_unique_value_limit):
      numerical_cols.append(col)


  print("Categorical Values (Nominal)")
  print(categorical_cols, end="\n\n")

  print("Categorical Values (Nominal)")
  print(numerical_cols)

  df_valid.isna().sum()
  from sklearn.impute import SimpleImputer

  imputer = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                      ("pandarizer",FunctionTransformer(lambda x: pd.DataFrame(x, columns = categorical_cols)))])
  df_categorical = imputer.fit_transform(df_valid[categorical_cols])

  imputer = Pipeline([("imputer", SimpleImputer(strategy="mean")),
                      ("pandarizer",FunctionTransformer(lambda x: pd.DataFrame(x, columns = numerical_cols)))])
  df_numerical = imputer.fit_transform(df_valid[numerical_cols])

  df_clean = df_categorical.join(df_numerical)

  df_clean.isna().sum()
  if year == "2013":
    df_clean.rename(columns = {'AREA':'Area in Square meter'}, inplace = True)
    df_clean

  if year == "2012":
    df_clean.rename(columns = {'Computed Area':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'PRODUCTION':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] / 1000)
    df_clean

  if year == "2011":
    df_clean.rename(columns = {'Area in Hectare':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'PRODUCTION IN QUINTALS':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
    df_clean

  if year == "2010":
    df_clean.rename(columns = {'AREAH':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'PRODUCTION IN QUINTALS':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
    df_clean

  if year == "2009":
    df_clean.rename(columns = {'Crop or Landuse':'CROP'}, inplace = True)
    df_clean.rename(columns = {'Area in hectare':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
    df_clean

  if year == "2008":
    df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
    df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean

  if year == "2007":
    df_clean.rename(columns = {'Area in Hectare':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
    df_clean

  if year == "2006":
    df_clean.rename(columns = {'Killil':'Region'}, inplace = True)
    df_clean.rename(columns = {'Wereda':'District'}, inplace = True)
    df_clean.rename(columns = {'Area in hectar':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean["Area in Square meter"] = (df_clean["Area in Square meter"] * 10000)
    df_clean

  if year == "2005":
    df_clean.rename(columns = {'Killil':'Region'}, inplace = True)
    df_clean.rename(columns = {'Wereda':'District'}, inplace = True)
    df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
    df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'Production in Quintal':'Production Of Condition in Kg'}, inplace = True)
    df_clean["Production Of Condition in Kg"] = (df_clean["Production Of Condition in Kg"] * 100)
    df_clean

  if year == "2004":
    df_clean.rename(columns = {'District(Wereda)':'District'}, inplace = True)
    df_clean.rename(columns = {'Crop or Land Use':'CROP'}, inplace = True)
    df_clean.rename(columns = {'Area (sq. m.)':'Area in Square meter'}, inplace = True)
    df_clean.rename(columns = {'Dry Weight Production (kg.)':'Production Of Condition in Kg'}, inplace = True)
    df_clean


  df_1 = df_clean[df_clean["CROP"] == 1]
  df_2 = df_clean[df_clean["CROP"] == 2]
  df_6 = df_clean[df_clean["CROP"] == 6]
  df_7 = df_clean[df_clean["CROP"] == 7]
  df_8 = df_clean[df_clean["CROP"] == 8]

  print(crop_1, df_1.shape)
  print(crop_2, df_2.shape)
  print(crop_6, df_6.shape)
  print(crop_7, df_7.shape)
  print(crop_8, df_8.shape)
  print("\nTotal:", df_1.shape[0] + df_2.shape[0] + df_6.shape[0] + df_7.shape[0] + df_8.shape[0])
  df_1.to_csv(f"{path}Clean_Meher_{year}_{crop_1}.csv")
  df_2.to_csv(f"{path}Clean_Meher_{year}_{crop_2}.csv")
  df_6.to_csv(f"{path}Clean_Meher_{year}_{crop_6}.csv")
  df_7.to_csv(f"{path}Clean_Meher_{year}_{crop_7}.csv")
  df_8.to_csv(f"{path}Clean_Meher_{year}_{crop_8}.csv")
  print(year)
  print(2*f"{80*'* '}\n")

Categorical Values (Nominal)
['Region', 'Zone', 'District', 'Farmers Association', 'Enumeration Area', 'Head sex', 'Holder id', 'Parcel', 'Field', 'Field Type', 'CROP', 'Ownership', 'Extenstion Program', 'Irrigation used?', 'Is the Soil Erosion', 'Measures taken to prevent Soil Erosion', 'SEED TYPE', 'Is there Damage?', 'Damage in Percent', 'Measure taken to prevent damage', 'Damage measure taken type', 'Fertilizer Used?', 'Fertilizer Type', 'How many times do you produce crops', 'What was the field used for?', 'APERCENT']

Categorical Values (Nominal)
['Household Id', 'Sampling Weight', 'Weight of non-improved seed', 'If Chemical Fertilizer,Quantity in KG', 'CERROR', 'AREAH', 'PRODUCTION IN QUINTALS', 'Fertilizer Quantity']
Barley (11524, 34)
Maize (18480, 34)
Sorghum (13848, 34)
Teff (14431, 34)
Wheat (11748, 34)

Total: 70031
2010
* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 